In [5]:
import random
import math

In [6]:
def add(v,w):
    """Adds corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"

    return [v_i + w_i for v_i, w_i in zip(v, w)]


def subtract(v, w):
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"

    return [v_i - w_i for v_i, w_i in zip(v, w)]

def vector_sum(vectors):
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided!"

    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"

    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]

def scalar_multiply(c,v):
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

def vector_mean(vectors):
    """Computes the element-wise average"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def dot(v, w):
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"
    #print(v)
    #print(w)
    return sum(v_i * w_i for v_i, w_i in zip(v, w))
def partial_difference_quotient(f,v,i,h):
        """Returns the i-th partial difference quotient of f at v"""
        w = [v_j + (h if j == i else 0)    # add h to just the ith element of v
             for j, v_j in enumerate(v)]
    
        return (f(w) - f(v)) / h
    
def estimate_gradient(f,v,h = 0.0001):
    return [partial_difference_quotient(f, v, i, h)
            for i in range(len(v))]

def gradient_step(v, gradient, step_size):
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

def sigmoid(t):
    return 1 / (1 + math.exp(-t))

def neuron_output(weights, inputs):
    # weights includes the bias term, inputs includes a 1
    #print(inputs)
    #print(weights)
    return sigmoid(dot(weights, inputs))

In [7]:
def create_layers(size,num_layers):
    """
    Create an array of random weights
    size = number of weights in each layer
    """
    network = []
    for i in range(num_layers):
        network.append([random.random() for _ in range(size + 1)])
    return network   

In [8]:
def sqerror_gradients(network,
                      input_vector,
                      target_vector):
    """
    Given a neural network, an input vector, and a target vector,
    make a prediction and compute the gradient of the squared error
    loss with respect to the neuron weights.
    """
    # forward pass
    hidden_outputs, outputs = feed_forward(network, input_vector)

    # gradients with respect to output neuron pre-activation outputs
    output_deltas = [output * (1 - output) * (output - target)
                     for output, target in zip(outputs, target_vector)]

    # gradients with respect to output neuron weights
    output_grads = [[output_deltas[i] * hidden_output
                     for hidden_output in hidden_outputs + [1]]
                    for i, output_neuron in enumerate(network[-1])]

    # gradients with respect to hidden neuron pre-activation outputs
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                         dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # gradients with respect to hidden neuron weights
    hidden_grads = [[hidden_deltas[i] * input for input in input_vector + [1]]
                    for i, hidden_neuron in enumerate(network[0])]

    return [hidden_grads, output_grads]

def training_network(xs,ys,network,learning_rate = 1.0):    
    for epoch in range(20000):
            for x, y in zip(xs, ys):
                gradients = sqerror_gradients(network, x, y)
    
            # Take a gradient step for each neuron in each layer
                network = [[gradient_step(neuron, grad, -learning_rate)
                            for neuron, grad in zip(layer, layer_grad)]
                           for layer, layer_grad in zip(network, gradients)]
    return network

In [9]:
def data_split(data,percent):
    """
    Split data based on percentage for testing/training networks
    """
    training = data[:int(len(data) * percent)]
    test = data[-int(len(data) * (1 - percent)):]
    return training,test

In [10]:
def feed_forward(neural_network,
                 input_vector):
    """
    Feeds the input vector through the neural network.
    Returns the outputs of all layers (not just the last one).
    """
    outputs = []

    for layer in neural_network:
        input_with_bias = input_vector + [1]              # Add a constant.
        output = [neuron_output(neuron, input_with_bias)  # Compute the output
                  for neuron in layer]                    # for each neuron.
        outputs.append(output)                            # Add to results.

        # Then the input to the next layer is the output of this one
        input_vector = output

    return outputs

Run test to make sure neural net is operating properly

In [11]:
def test_neuralnet():
    """
    A quick test to make sure we can train the NN
    We're training it on learning XOR since its fast, simple,
    and if something goes wrong we can troubleshoot easy
    """
    full_network = []
    hidden_layer = create_layers(2,2)
    output_layer = create_layers(2,1)
    full_network.append(hidden_layer)
    full_network.append(output_layer)
    #test data
    xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
    ys = [[0.], [1.], [1.], [0.]]
    network = training_network(xs,ys,full_network)
    #lets make sure everything is working
    print("Running assert block")
    assert feed_forward(network, [0, 0])[-1][0] < 0.01
    assert feed_forward(network, [0, 1])[-1][0] > 0.99
    assert feed_forward(network, [1, 0])[-1][0] > 0.99
    assert feed_forward(network, [1, 1])[-1][0] < 0.01
    print("Test complete!")

In [12]:
test_neuralnet()

Running assert block


AssertionError: 

In [13]:
network = training_network(xs,ys,full_network)

NameError: name 'xs' is not defined

In [14]:
print()
feed_forward(network,[0,1])[-1][0]
print(feed_forward(network,[1,0])[-1][0])
print(feed_forward(network,[1,1])[-1][0])

SyntaxError: invalid syntax (<ipython-input-14-7ab52a7a4b04>, line 4)

In [15]:
import pandas as pd

In [54]:
df = pd.read_csv("small.csv")
df.head()

,accountAgeDays,numItems,localTime,paymentMethodAgeDays,paymentMethod_creditcard,paymentMethod_paypal,paymentMethod_storecredit,label
16865,2,1,4.886641,1.15,1,0,0,0
24140,10,1,4.461622,0.00,1,0,0,0
4346,2,1,5.040929,0.00,1,0,0,0
25366,126,1,4.921349,0.00,1,0,0,0
19207,1696,1,4.836982,0.00,1,0,0,0


In [55]:
training_set = df.sample(frac = 0.6)
test_set = df.drop(training_set.index)
test_set = test_set.reset_index()
training_set = training_set.reset_index()

In [56]:
Xs = []
Ys = []

In [57]:
columns = training_set.columns.tolist()
print(columns[1:-1])

['accountAgeDays', 'numItems', 'localTime', 'paymentMethodAgeDays', 'paymentMethod_creditcard', 'paymentMethod_paypal', 'paymentMethod_storecredit']


In [58]:
columns = training_set.columns.tolist()
for i in range(len(training_set)):
    Xs.append(training_set.loc[i, columns[1:-1]].values.flatten().tolist())
for i in range(len(training_set)):
    Ys.append(training_set.loc[i,columns[-1]].flatten().tolist())

In [ ]:
print(Xs)

In [59]:
print(len(Xs))
print(len(Ys))
print(len(training_set))

229
229
229


In [33]:
full_network = []
hidden_layer = create_layers(7,7)
output_layer = create_layers(7,1)
full_network.append(hidden_layer)
full_network.append(output_layer)
print(len(full_network[1]))
print(len(Xs))
print(len(Ys))

1
229
229


In [62]:
network = training_network(Xs,Ys,full_network)

In [38]:
test_x = []
test_y = []
columns = test_set.columns.tolist()
for i in range(len(test_set)):
    test_x.append(test_set.loc[i, columns[1:-1]].values.flatten().tolist())
for i in range(len(test_set)):
    test_y.append(test_set.loc[i,columns[-1]].flatten().tolist())

In [44]:
positive = 0
actual_positive = 0
for item in test_x:
    if feed_forward(network,item)[-1][0] > 0.01:
        positive += 1
for item in test_y:
    if 1 in item:
        actual_positive += 1

In [45]:
print(test_y)

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0]]


In [46]:
print("Calc positive: " + str(positive))
print("Actual positive: " + str(actual_positive))

Calc positive: 1
Actual positive: 1


In [50]:
final_x = []
final_y = []
columns = df2.columns.tolist()
for i in range(len(df2)):
    final_x.append(df2.loc[i, columns[:-1]].values.flatten().tolist())
for i in range(len(df2)):
    final_y.append(df2.loc[i,columns[-1]].flatten().tolist())

In [51]:
final_x[0]

[2.0, 1.0, 4.886641, 1.15, 1.0, 0.0, 0.0]

In [52]:
positive = 0
actual_positive = 0
for item in final_x:
    if feed_forward(network,item)[-1][0] > 0.01:
        positive += 1
for item in final_y:
    if 1 in item:
        actual_positive += 1
print("Calc positive: " + str(positive))
print("Actual positive: " + str(actual_positive))

Calc positive: 3
Actual positive: 3


In [ ]:
train,test = 

In [ ]:
data = [1,2,3,4,5,6,7,8,9,0]
training = data[: int(len(data)*0.6)]
test = data[-int(len(data)*0.4):]
print(training)
print(test)